In [23]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt


In [ ]:
## dataloader

In [2]:
## train


for step, data in enumerate(dataloader) :

    data

    pred = model(x) 
    loss = criterion(pred, gt)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()




IndentationError: unexpected indent (3295619247.py, line 4)

dict_keys(['name', 'attributes', 'timestamp', 'labels', 'imgsize'])


In [18]:
categories = []

with open('data/label/det_train.json') as f :
    json_info = json.load(f)


for info in json_info :
    try :
        labels = info['labels']
    except :
        print(info['name'])
        continue
    for l in labels :
        categories.append(l['category'])
        

11ecaf4a-837e3550.jpg
272cd572-f7289b9d.jpg
282678b0-5f4e4eb3.jpg
31a83844-ba334636.jpg
321877a3-f277463d.jpg
48f20d4e-504d2377.jpg
49cf8611-8991f7a7.jpg
51a2ee54-e7f7d10f.jpg
57ea20aa-d836f65b.jpg
65c115f0-324deb97.jpg


In [21]:
from glob import glob

train_imgfiles = glob('data/images/train/*.jpg')

In [22]:
print(json_info[1000]['name'], train_imgfiles[1000])

02b7b9ee-74fb981c.jpg data/images/train\02b7b9ee-74fb981c.jpg


In [ ]:
class BDDDataset(Dataset) :
    def __init__(self, imgdir, jsonfile, transform = None ) :
        super().__init__()
        self.imgdir = imgdir
        self.imgfiles = glob(os.path.join(self.imgdir, '*.jpg'))
        
        with open(jsonfile, 'r') as f :
            json_infos = json.load(f)
        self.json_infos = json_infos

        ## make sure json information and imgfile are matched
        for info, imgfile in tqdm(zip(self.json_infos, self.imgfiles), desc = 'validate json and image matching...') :
            os.path.basename(imgfile) == info['name']

        self.transform = transform

    def __getitem__(self, idx) :
        imgfile = self.imgfiles[idx]
        json_info = self.json_infos[idx]

        json_info = json_info['labels'] if 'labels' in json_info.keys() else []

        image = plt.imread(imgfile)
        if self.transform :
            image = self.transform(image)

        ## bbox handling
        for info in json_info :
            label = info['category']
            coord = info['box2d']
            x, y = coord['x'], coord['y']
            w, h = coord['w'], coord['h']
        

        

    def __len__(self) :
        return len(self.imgfiles)


In [ ]:
class YOLO(nn.Module):
    def __init__(self, VGG16, num_grids = 7, numBox = 2, num_classes = 13):
        '''
        VGG16 : backbone network
        num_grids : # of grids to divde an image, make sure to be dividable with image size
        numBox : number of boxes per grid
        num_classes : # of classes. In BDD, there are 13 classes
        '''
        super().__init__()

        self.backbone = VGG16
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 512,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.Conv2d(in_channels = 1024,out_channels = 1024, kernel_size = 3, padding = 1),
            nn.LeakyReLU(),
            nn.Flatten()
        )
        self.linear = nn.Sequential(
            nn.Linear((num_grids**2) * 1024, 4096),
            nn.LeakyReLU(),
            nn.Dropout(),
            nn.Linear(4096, (num_grids**2)*(numBox ))
        )

        # 가중치 초기화
        for m in self.conv.modules():
    	    if isinstance(m, nn.Conv2d) :
		        nn.init.normal_(m.weight, mean=0, std=0.01)
                
        for m in self.linear.modules():
        	if isinstance(m, nn.Linear) :
                 nn.init.normal_(m.weight, mean=0, std=0.01)
                

    # 정전파 
    def forward(self, x):
        out = self.backbone(x)
        out = self.conv(out)
        out = self.linear(out)
        out = torch.reshape(out, (-1 ,7, 7, 30))
        return out

In [25]:
1470 / 7 / 7 / 30

1.0